# Golden SAML AD FS Mail Access Hunt

### Scope the Environment
- Identity Provider (IdP) -  On-Prem Active Directory Federation Services (AD FS) server w/ Azure AD Connect
- Cloud Service Provider (SP) - Microsoft 365 (Mail, Defender)
- On-Prem Workstations

### Dataset
https://securitydatasets.com/notebooks/compound/GoldenSAMLADFSMailAccess.html
- WindowsEvents.Zip
- Microsoft365DefenderEvents.Zip
- AADAuditEvents.Zip
- OfficeActivityEvents.Zip

#### Simple Data Ingestion

All data is unzipped and loaded to two SQlite stores (SQL-supported lakehouse simulation) using [kestrel-tool::mkdb](https://github.com/opencybersecurityalliance/kestrel-lang/blob/develop/packages/kestrel_tool/src/kestrel_tool/mkdb.py)

#### Simulated On-premise Datastore `sqlite:///onpremise.db`

| Data File | Datastore Index | Kestrel Datasource | ........................................................................................................................................................................ |
| :------- | :------ | :------ | ------- |
| WindowsEvents.Zip | windows | GoldenSAML-WindowsEvents ||

#### Simulated Cloud Datastore `sqlite:///cloud.db`

| Data File | Datastore Index | Kestrel Datasource | .............................................................................................................................. |
| :------- | :------ | :------ | ------- |
| Microsoft365DefenderEvents.Zip | msdefender | GoldenSAML-Microsoft365DefenderEvents||
| AADAuditEvents.Zip | aad | GoldenSAML-AADAuditEvents ||
| OfficeActivityEvents.Zip | office | GoldenSAML-OfficeActivityEvents||

### Campaign Hypothesis
Cyber Threat Actor (CTA) will compromise the organization's identity provider (AD FS) and attempt to obtain the private key used for signing SAML tokens, to then forge a valid SAML token that will impersonate a legitimate user to access sensitive data hosted by the organization's cloud service provider (Microsoft 365). 

#### Mitre Att&ck Mapping


## Steps in This Huntbook

1. Read the [GoldenSAML blog](https://www.cyberark.com/resources/threat-research-blog/golden-saml-newly-discovered-attack-technique-forges-authentication-to-cloud-apps) to understand the generic attack flow
2. Develop the threat hypothesis around the [SimuLand GoldenSAML attack simulation](https://simulandlabs.com/labs/GoldenSAML/README.html)
3. Using the blog, attack simulation, and Mitre mapping above, develop hunt analytics around:
    - [T1003 - OS Credential Dumping](https://attack.mitre.org/techniques/T1003/)
    - [T1087.002 - Account Discovery: Domain Account](https://attack.mitre.org/techniques/T1087/002/)
    - [T1098.002: Additional Email Delegate Permissions](https://attack.mitre.org/techniques/T1098/002/)
    - [T1134: Access Token Manipulation](https://attack.mitre.org/techniques/T1134/)
    - [T1552.004 - Unsecured Credentials: Private Keys](https://attack.mitre.org/techniques/T1552/004/)
4. Move across multiple data sources to verify different phases of the attack from multiple angles (already written in this huntbook)
5. **Execute this huntbook and report your findings**
6. Further drill down
    - Add new cells in this huntbook to explore other paths of the hunt
    - Explore other aspects of the attack with any *[Quiz]* huntbook in the same folder
    - Check the *Kestrel Execution Explained* huntbook to get a basic idea of the Kestrel abstraction

In [ ]:
# Let's first understand what data we have available in our datasets
# `type_uid` is a required attribute of OCSF event/activity
# type_uid LIKE '%' means any event

win_events = GET event FROM sqlalchemy://GoldenSAML-WindowsEvents
             WHERE type_uid LIKE '%'
             START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z

# We use customized OCSF field `event_id` for holding raw event id (if exist in source)
# We use OCSF field `type_name` for holding event name

DISP win_events ATTR event_id, type_name

In [ ]:
mde_events = GET event FROM sqlalchemy://GoldenSAML-Microsoft365DefenderEvents
             WHERE type_uid LIKE '%'
             START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z
 
DISP mde_events ATTR type_name

In [ ]:
aad_events = GET event FROM sqlalchemy://GoldenSAML-AADAuditEvents
             WHERE type_uid LIKE '%'
             START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z

DISP aad_events ATTR type_name

In [ ]:
office_events = GET event FROM sqlalchemy://GoldenSAML-OfficeActivityEvents
                WHERE type_uid LIKE '%'
                START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z

DISP office_events ATTR type_name

Now that we know what type of events we have available to use.  Let's see how these events apply to the attack
steps that were broken down in [Simulandlabs Golden SAML Overview](https://simulandlabs.com/labs/GoldenSAML/README.html).

The steps show that the actor would need to:
- Export the AD FS Configuration
- Export the AD FS DKM Master Key to decrypt the private key
- Export the AD FS Certificates
- Forge the SAML Tokens
- Get OAuth Access Token with SAML Assertion
- Finally, access mail messages using MS Graph


A few of these steps we may not be able to see because they can be conducted locally on the attacker's system.  Let's see what we can identify!!

Let's start by looking for any activity around the AD FS configuration.  Doing a quick Google search shows us that
[configuration](https://learn.microsoft.com/en-us/windows-server/identity/ad-fs/technical-reference/the-role-of-the-ad-fs-configuration-database) resides in the Windows Internal Database (WID) or a SQL Server.

What events can we see that might point us towards a database?

Windows Event ID [33205](https://www.ultimatewindowssecurity.com/sqlserver/auditlog/sampleevent.aspx) shows us SQL Server Audit events.

In [ ]:
# Let's write a query to view all events from Event ID 33205

sql_events = GET event FROM sqlalchemy://GoldenSAML-WindowsEvents
             WHERE event_id = 33205

# an alternative way is to use filter from our previous variable `win_events`
#sql_events = win_events WHERE event_id = 33205
             
DISP sql_events

There is not a lot of information from this event but we can see that some action happened on our `adfs01` host.

Let's see if there are other events around that host. 

In [ ]:
procs = GET process FROM sqlalchemy://GoldenSAML-WindowsEvents
        WHERE endpoint.hostname LIKE "%adfs01%"

# an alternative way is to use FIND from our previous variable `win_events`
#procs = FIND process RESPONDED win_events
#        WHERE endpoint.hostname LIKE "%adfs01%"

DISP procs ATTR name, pid, endpoint.hostname

There is a powershell process that's running on `adfs01`. Let's see what that process is doing.

In [ ]:
powershell_event = FIND event RESPONDED BY procs

DISP powershell_event

This PowerShell process is running a query to the `WID` on `ADFS01`.  Someone is interacting with our WID database!
Let's see if defender caught any activity from this process id.

In [ ]:
powershell_mde_events = GET event FROM sqlalchemy://GoldenSAML-Microsoft365DefenderEvents
                        WHERE actor.process.pid = procs.pid
                          AND device.hostname = procs.endpoint.hostname

DISP powershell_mde_events ATTR time, type_name, device.hostname SORT BY time ASC


mde_actor_procs = FIND process ORIGINATED mde_events
DISP mde_actor_procs


mde_actor_users = FIND user ORIGINATED mde_events
DISP mde_actor_users

Using the PowerShell `PID`, we are able to see that there are also `ldapsearch` queries executing on `ADFS01` and who is executing those queries. What are those `ldapsearch` events doing?

In [ ]:
ldap_query = FIND query_info RESPONDED BY powershell_mde_events

DISP ldap_query

We see a few things that are interesting from the `attr_list`.  There is a mention of `thumbnailphoto`, where if we do a quick Google search for `adfs` and `thumbnailphoto`, we can see that this is a way from the actor to access the [Distributed Key Manager](https://threathunterplaybook.com/library/windows/adfs_dkm_keys.html) (DKM) Keys. 

Looking back at the steps that the actor needs to accomplish, we can see evidence that the actor has moved to Step 2.  There is an attribute value listed in the `search_filter` as well as searching for `users` in the `Domain Admins` group.  We may be able to hunt for more activity based on these items.


So far we have found activity for:
  - T1087.002 - Account Discovery: Domain Account
  - T1552.004 - Unsecured Credentials: Private Keys
  
  
Let's take a look at our win events for anything that might catch ADFS objects since the attribute value might be an object value.

In [ ]:
thumbnail_object = GET entity FROM sqlalchemy://GoldenSAML-WindowsEvents
                   WHERE uid LIKE '%9736f74f-fd37-4b02-80e8-8120a72ad6c2%'

# an alternative way is to use FIND from our previous variable `win_events`
#thumbnail_object = FIND managed_entity RESPONDED win_events
#                   WHERE uid LIKE '%9736f74f-fd37-4b02-80e8-8120a72ad6c2%'

DISP thumbnail_object

thumbnail_user = FIND user READ thumbnail_object
DISP thumbnail_user

thumbnail_event = FIND event RESPONDED BY thumbnail_object
DISP thumbnail_event ATTR time, event_id, type_uid, type_name, device.hostname SORT BY time ASC

Looks like we see activity that is based off of the previous AD Object.  Windows Event ID `4662` shows any time an AD object was accessed and from what system that activity took place on.  If we look at the `entity.data` this looks like guids from additional objects and the range looks like this might be enumeration.  For how quickly these calls are being made, this looks very automated.  What tool might be able to do this type of activity on a DC?  

We have found activity for:
  - T1003 - OS Credential Dumping
  
  
We also have a rough time of the start of the attack `2021-08 13:11:00`

What other events happened around that time?

Let's see what was captured by MDE?

In [ ]:
additional_events = mde_events
                    WHERE (device.hostname LIKE '%dc01%' OR device.hostname LIKE '%adfs01%')
                      AND type_name != "LdapSearch" # exclude the LdapSearch we already explored

DISP additional_events

During the ldapsearch we see that there was a user with details that were changed?

Let's zoom into the strange part:

In [ ]:
additional_events_actor = FIND user ORIGINATED additional_events
DISP additional_events_actor

additional_events_user = FIND user RESPONDED additional_events
DISP additional_events_user

Let's follow that trail around to the other datasources.

In [ ]:
aad_related_events = GET event FROM sqlalchemy://GoldenSAML-AADAuditEvents
                     WHERE actor.user.uid = additional_events_actor.uid
                        OR actor.user.uid = additional_events_user.uid

DISP aad_related_events

We see that user doing interesting Application actions with a user_agent showing PowerShell.  Could this be related to the other malicious activity observed using PowerShell?  There are also privileges added and referencing Microsoft Graph.  Why would a user access their own mailbox with an API? Let's see if those privileges were successfully used.

We have found activity for:
  - T1098.002: Additional Email Delegate Permissions
  - T1134: Access Token Manipulation

In [ ]:
priv_mde = mde_events WHERE privileges LIKE "%Mail.ReadWrite%"

DISP priv_mde SORT BY time ASC

Finally, Let's see if the application was access

In [ ]:
aad_users = FIND user ORIGINATED aad_related_events

DISP aad_users

uid_office = GET event
             FROM sqlalchemy://GoldenSAML-OfficeActivityEvents
             WHERE actor.user.name = aad_users.name

DISP uid_office ATTR time, src_endpoint.ip, type_name, entity.type, entity.data, actor.user.uid, actor.user.name